In [1]:
#Import de bibliotecas a serem utilizdas
import pandas as pd
import numpy as np
import pymssql
from sqlalchemy.engine import URL,create_engine
from sqlalchemy_utils import database_exists, create_database

## Leitura do Banco de dados

In [2]:
#Criação de conexão ao banco de dados para leitura
server_name = "DESKTOP-9RK4J8O"
db_name = "AdventureWorksDW2019"
connection_str =  'Driver={SQL Server};' + 'Server=' + server_name + ";Database=" + db_name + ";Trusted_Connection=yes;"

connection_url = URL.create("mssql+pyodbc", query={"odbc_connect": connection_str})
engine = create_engine(connection_url)
con = engine.connect()

In [3]:
#Leitura do banco de dados em um dataframe do pandas
FactFinance = pd.read_sql("SELECT * FROM dbo.FactFinance", con)
FactFinance.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39409 entries, 0 to 39408
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   FinanceKey          39409 non-null  int64         
 1   DateKey             39409 non-null  int64         
 2   OrganizationKey     39409 non-null  int64         
 3   DepartmentGroupKey  39409 non-null  int64         
 4   ScenarioKey         39409 non-null  int64         
 5   AccountKey          39409 non-null  int64         
 6   Amount              39409 non-null  float64       
 7   Date                39409 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(1), int64(6)
memory usage: 2.4 MB


In [4]:
#Leitura das demais tabelas
DimAccount = pd.read_sql("SELECT * FROM dbo.DimAccount", con)
DimDate = pd.read_sql("SELECT * FROM dbo.DimDate", con)
DimDepartmentGroup = pd.read_sql("SELECT * FROM dbo.DimDepartmentGroup", con)
DimOrganization = pd.read_sql("SELECT * FROM dbo.DimOrganization", con)
DimScenario = pd.read_sql("SELECT * FROM dbo.DimScenario", con)
DimCurrency = pd.read_sql("SELECT * FROM dbo.DimCurrency", con)

In [5]:
#Verificando o nomes das colunas das tabelas carregadas
tabelas = [FactFinance,DimAccount,DimCurrency, DimOrganization,DimDate,DimDepartmentGroup,DimOrganization,DimScenario,DimCurrency]
for df in tabelas:
    print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39409 entries, 0 to 39408
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   FinanceKey          39409 non-null  int64         
 1   DateKey             39409 non-null  int64         
 2   OrganizationKey     39409 non-null  int64         
 3   DepartmentGroupKey  39409 non-null  int64         
 4   ScenarioKey         39409 non-null  int64         
 5   AccountKey          39409 non-null  int64         
 6   Amount              39409 non-null  float64       
 7   Date                39409 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(1), int64(6)
memory usage: 2.4 MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99 entries, 0 to 98
Data columns (total 10 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   AccountKey                    

## DimCurrency

In [6]:
DimCurrency.head()

,CurrencyKey,CurrencyAlternateKey,CurrencyName
0,1,AFA,Afghani
1,2,DZD,Algerian Dinar
2,3,ARS,Argentine Peso
3,4,AMD,Armenian Dram
4,5,AWG,Aruban Guilder


In [7]:
DimOrganization.head()

,OrganizationKey,ParentOrganizationKey,PercentageOfOwnership,OrganizationName,CurrencyKey
0,1,NaN,1,AdventureWorks Cycle,100
1,2,1.0,1,North America Operations,100
2,3,14.0,1,Northeast Division,100
3,4,14.0,1,Northwest Division,100
4,5,14.0,1,Central Division,100


#### Irei utilizar apenas o nome da moeda em meu relatório, portanto irei levar essa informação da tabela DimCurrency para a DimOrganization a fim de simplificar o modelo e remover registros desnecessários

In [8]:
#Trazer os dados da currency para organization
DimOrganization_trat = DimOrganization.merge(DimCurrency, how='left', on='CurrencyKey')
DimOrganization_trat.head()

,OrganizationKey,ParentOrganizationKey,PercentageOfOwnership,OrganizationName,CurrencyKey,CurrencyAlternateKey,CurrencyName
0,1,NaN,1,AdventureWorks Cycle,100,USD,US Dollar
1,2,1.0,1,North America Operations,100,USD,US Dollar
2,3,14.0,1,Northeast Division,100,USD,US Dollar
3,4,14.0,1,Northwest Division,100,USD,US Dollar
4,5,14.0,1,Central Division,100,USD,US Dollar


In [9]:
#Fitrando colunas
DimOrganization_trat = DimOrganization_trat[["OrganizationKey","ParentOrganizationKey","OrganizationName","CurrencyName"]]
DimOrganization_trat.head()

,OrganizationKey,ParentOrganizationKey,OrganizationName,CurrencyName
0,1,NaN,AdventureWorks Cycle,US Dollar
1,2,1.0,North America Operations,US Dollar
2,3,14.0,Northeast Division,US Dollar
3,4,14.0,Northwest Division,US Dollar
4,5,14.0,Central Division,US Dollar


## DimOrganization

In [10]:
#Mapeando as hierarquias
OrgHier = DimOrganization_trat[["OrganizationKey","ParentOrganizationKey"]]
OrgHier = OrgHier.rename(columns={"OrganizationKey":"ParentOrganizationKey","ParentOrganizationKey":"ParentOrganizationKey2"})
OrgHier.head()

,ParentOrganizationKey,ParentOrganizationKey2
0,1,NaN
1,2,1.0
2,3,14.0
3,4,14.0
4,5,14.0


In [11]:
#Acrescentando hierarquias em colunas
DimOrganization_trat2 = DimOrganization_trat.merge(OrgHier, how='left', on='ParentOrganizationKey')
DimOrganization_trat2.head()

,OrganizationKey,ParentOrganizationKey,OrganizationName,CurrencyName,ParentOrganizationKey2
0,1,NaN,AdventureWorks Cycle,US Dollar,NaN
1,2,1.0,North America Operations,US Dollar,NaN
2,3,14.0,Northeast Division,US Dollar,2.0
3,4,14.0,Northwest Division,US Dollar,2.0
4,5,14.0,Central Division,US Dollar,2.0


In [12]:
#Acrescentando hierarquias em colunas
OrgHier = OrgHier.rename(columns={"ParentOrganizationKey":"ParentOrganizationKey2","ParentOrganizationKey2":"ParentOrganizationKey3"})
DimOrganization_trat3 = DimOrganization_trat2.merge(OrgHier, how='left', on='ParentOrganizationKey2')
DimOrganization_trat3

,OrganizationKey,ParentOrganizationKey,OrganizationName,CurrencyName,ParentOrganizationKey2,ParentOrganizationKey3
0,1,NaN,AdventureWorks Cycle,US Dollar,NaN,NaN
1,2,1.0,North America Operations,US Dollar,NaN,NaN
2,3,14.0,Northeast Division,US Dollar,2.0,1.0
3,4,14.0,Northwest Division,US Dollar,2.0,1.0
4,5,14.0,Central Division,US Dollar,2.0,1.0
5,6,14.0,Southeast Division,US Dollar,2.0,1.0
6,7,14.0,Southwest Division,US Dollar,2.0,1.0
7,8,2.0,Canadian Division,Canadian Dollar,1.0,NaN
8,9,1.0,European Operations,EURO,NaN,NaN
9,10,1.0,Pacific Operations,Australian Dollar,NaN,NaN


In [13]:
#Identificar quais codigos de org tem na fato
regOrg = FactFinance.OrganizationKey.unique()
#Filtrando registros da Organization que tem na fato
DimOrganization_trat4 = DimOrganization_trat3[DimOrganization_trat3['OrganizationKey'].isin(regOrg)]
DimOrganization_trat4

,OrganizationKey,ParentOrganizationKey,OrganizationName,CurrencyName,ParentOrganizationKey2,ParentOrganizationKey3
2,3,14.0,Northeast Division,US Dollar,2.0,1.0
3,4,14.0,Northwest Division,US Dollar,2.0,1.0
4,5,14.0,Central Division,US Dollar,2.0,1.0
5,6,14.0,Southeast Division,US Dollar,2.0,1.0
6,7,14.0,Southwest Division,US Dollar,2.0,1.0
7,8,2.0,Canadian Division,Canadian Dollar,1.0,NaN
10,11,9.0,France,EURO,1.0,NaN
11,12,9.0,Germany,EURO,1.0,NaN
12,13,10.0,Australia,Australian Dollar,1.0,NaN


In [14]:
#Separando os registros para ajuste de hierarquia
DimOrganization_trat5 = DimOrganization_trat4[DimOrganization_trat4['ParentOrganizationKey3'].notna()]
DimOrganization_trat5 = DimOrganization_trat5.rename(columns={"ParentOrganizationKey":"CountryName","OrganizationName":"DivisionName","ParentOrganizationKey2":"OperationName","ParentOrganizationKey3":"CycleName"})
DimOrganization_trat5.head()

,OrganizationKey,CountryName,DivisionName,CurrencyName,OperationName,CycleName
2,3,14.0,Northeast Division,US Dollar,2.0,1.0
3,4,14.0,Northwest Division,US Dollar,2.0,1.0
4,5,14.0,Central Division,US Dollar,2.0,1.0
5,6,14.0,Southeast Division,US Dollar,2.0,1.0
6,7,14.0,Southwest Division,US Dollar,2.0,1.0


In [15]:
#Separando os registros para ajuste de hierarquia
DimAux = DimOrganization_trat4[DimOrganization_trat4['ParentOrganizationKey3'].isna()]
DimAux = DimAux.rename(columns={"ParentOrganizationKey":"OperationName","OrganizationName":"CountryName","ParentOrganizationKey2":"CycleName","ParentOrganizationKey3":"DivisionName"})
DimAux.head()

,OrganizationKey,OperationName,CountryName,CurrencyName,CycleName,DivisionName
7,8,2.0,Canadian Division,Canadian Dollar,1.0,NaN
10,11,9.0,France,EURO,1.0,NaN
11,12,9.0,Germany,EURO,1.0,NaN
12,13,10.0,Australia,Australian Dollar,1.0,NaN


In [16]:
#Junção dos registros
DimOrganization_trat6 = pd.concat([DimOrganization_trat5,DimAux])
DimOrganization_trat6

,OrganizationKey,CountryName,DivisionName,CurrencyName,OperationName,CycleName
2,3,14.0,Northeast Division,US Dollar,2.0,1.0
3,4,14.0,Northwest Division,US Dollar,2.0,1.0
4,5,14.0,Central Division,US Dollar,2.0,1.0
5,6,14.0,Southeast Division,US Dollar,2.0,1.0
6,7,14.0,Southwest Division,US Dollar,2.0,1.0
7,8,Canadian Division,NaN,Canadian Dollar,2.0,1.0
10,11,France,NaN,EURO,9.0,1.0
11,12,Germany,NaN,EURO,9.0,1.0
12,13,Australia,NaN,Australian Dollar,10.0,1.0


In [17]:
#Substituindo os códigos pelos nomes
OrgName = pd.Series(DimOrganization_trat.OrganizationName.values,index=DimOrganization_trat.OrganizationKey).to_dict()
DimOrganization_trat7 = DimOrganization_trat6.replace({"CountryName": OrgName,"OperationName": OrgName,"CycleName": OrgName})
DimOrganization_trat7

,OrganizationKey,CountryName,DivisionName,CurrencyName,OperationName,CycleName
2,3,USA Operations,Northeast Division,US Dollar,North America Operations,AdventureWorks Cycle
3,4,USA Operations,Northwest Division,US Dollar,North America Operations,AdventureWorks Cycle
4,5,USA Operations,Central Division,US Dollar,North America Operations,AdventureWorks Cycle
5,6,USA Operations,Southeast Division,US Dollar,North America Operations,AdventureWorks Cycle
6,7,USA Operations,Southwest Division,US Dollar,North America Operations,AdventureWorks Cycle
7,8,Canadian Division,NaN,Canadian Dollar,North America Operations,AdventureWorks Cycle
10,11,France,NaN,EURO,European Operations,AdventureWorks Cycle
11,12,Germany,NaN,EURO,European Operations,AdventureWorks Cycle
12,13,Australia,NaN,Australian Dollar,Pacific Operations,AdventureWorks Cycle


In [18]:
#Reordenando
DimOrganization_trat7 = DimOrganization_trat7[["OrganizationKey","DivisionName","CountryName","OperationName","CycleName","CurrencyName"]]
DimOrganization_trat7.head()

,OrganizationKey,DivisionName,CountryName,OperationName,CycleName,CurrencyName
2,3,Northeast Division,USA Operations,North America Operations,AdventureWorks Cycle,US Dollar
3,4,Northwest Division,USA Operations,North America Operations,AdventureWorks Cycle,US Dollar
4,5,Central Division,USA Operations,North America Operations,AdventureWorks Cycle,US Dollar
5,6,Southeast Division,USA Operations,North America Operations,AdventureWorks Cycle,US Dollar
6,7,Southwest Division,USA Operations,North America Operations,AdventureWorks Cycle,US Dollar


## DimDepartment

In [19]:
DimDepartmentGroup.head()

,DepartmentGroupKey,ParentDepartmentGroupKey,DepartmentGroupName
0,1,NaN,Corporate
1,2,1.0,Executive General and Administration
2,3,1.0,Inventory Management
3,4,1.0,Manufacturing
4,5,1.0,Quality Assurance


### Verifico que a DimDepartmentGroup também possui o código do "pai" invés do nome

In [20]:
#Trazer o nome dos grupos de departamentos 'pais' invés do código
DimDepartmentGroup_trat = DimDepartmentGroup.merge(DimDepartmentGroup, how='left', left_on='ParentDepartmentGroupKey',right_on='DepartmentGroupKey')
DimDepartmentGroup_trat

,DepartmentGroupKey_x,ParentDepartmentGroupKey_x,DepartmentGroupName_x,DepartmentGroupKey_y,ParentDepartmentGroupKey_y,DepartmentGroupName_y
0,1,NaN,Corporate,NaN,NaN,NaN
1,2,1.0,Executive General and Administration,1.0,NaN,Corporate
2,3,1.0,Inventory Management,1.0,NaN,Corporate
3,4,1.0,Manufacturing,1.0,NaN,Corporate
4,5,1.0,Quality Assurance,1.0,NaN,Corporate
5,6,1.0,Research and Development,1.0,NaN,Corporate
6,7,1.0,Sales and Marketing,1.0,NaN,Corporate


In [21]:
#Seleção das colunas
DimDepartmentGroup_trat2 = DimDepartmentGroup_trat[["DepartmentGroupKey_x","DepartmentGroupName_x","DepartmentGroupName_y"]]
DimDepartmentGroup_trat2.head()

,DepartmentGroupKey_x,DepartmentGroupName_x,DepartmentGroupName_y
0,1,Corporate,NaN
1,2,Executive General and Administration,Corporate
2,3,Inventory Management,Corporate
3,4,Manufacturing,Corporate
4,5,Quality Assurance,Corporate


In [22]:
#Renomeação das colunas
DimDepartmentGroup_trat3 = DimDepartmentGroup_trat2.rename(columns={"DepartmentGroupKey_x":"DepartmentGroupKey","DepartmentGroupName_x":"DepartmentGroupName","DepartmentGroupName_y":"ParentDepartmentGroupName"})
DimDepartmentGroup_trat3.head()

,DepartmentGroupKey,DepartmentGroupName,ParentDepartmentGroupName
0,1,Corporate,NaN
1,2,Executive General and Administration,Corporate
2,3,Inventory Management,Corporate
3,4,Manufacturing,Corporate
4,5,Quality Assurance,Corporate


In [23]:
DimDepartmentGroup_trat3.loc[0] = [1,np.nan,'Corporate']
DimDepartmentGroup_trat3

,DepartmentGroupKey,DepartmentGroupName,ParentDepartmentGroupName
0,1,NaN,Corporate
1,2,Executive General and Administration,Corporate
2,3,Inventory Management,Corporate
3,4,Manufacturing,Corporate
4,5,Quality Assurance,Corporate
5,6,Research and Development,Corporate
6,7,Sales and Marketing,Corporate


## DimAccount

In [24]:
DimAccount.head()

,AccountKey,ParentAccountKey,AccountCodeAlternateKey,ParentAccountCodeAlternateKey,AccountDescription,AccountType,Operator,CustomMembers,ValueType,CustomMemberOptions
0,1,NaN,1,NaN,Balance Sheet,None,~,None,Currency,None
1,2,1.0,10,1.0,Assets,Assets,+,None,Currency,None
2,3,2.0,110,10.0,Current Assets,Assets,+,None,Currency,None
3,4,3.0,1110,110.0,Cash,Assets,+,None,Currency,None
4,5,3.0,1120,110.0,Receivables,Assets,+,None,Currency,None


In [25]:
#Removeção de colunas desnecessárias pois não trazem informação
DimAccount_trat = DimAccount[['AccountKey','ParentAccountKey','AccountDescription','AccountType','Operator','ValueType']]
DimAccount_trat.head()

,AccountKey,ParentAccountKey,AccountDescription,AccountType,Operator,ValueType
0,1,NaN,Balance Sheet,None,~,Currency
1,2,1.0,Assets,Assets,+,Currency
2,3,2.0,Current Assets,Assets,+,Currency
3,4,3.0,Cash,Assets,+,Currency
4,5,3.0,Receivables,Assets,+,Currency


### Desconsiderarei registros da fato onde a account tenha operator igual a ~ ou ValueType igual a units

In [26]:
#Identificar quais códigos de conta tem operador ~
regRemovOpe = DimAccount.AccountKey[DimAccount['Operator'] == '~'].tolist()
#Removendo na fato os registros referentes a estas accounts
FactFinance_trat = FactFinance[~FactFinance['AccountKey'].isin(regRemovOpe)]

In [27]:
#Identificar quais códigos de conta tem ValueType igual a units
regRemovTyp = DimAccount.AccountKey[DimAccount['ValueType'] == 'Units'].tolist()
#Removendo na fato os registros referentes a estas accounts
FactFinance_trat2 = FactFinance_trat[~FactFinance_trat['AccountKey'].isin(regRemovTyp)]

In [28]:
#Mapeando as hierarquias
AccHier = DimAccount_trat[["AccountKey","ParentAccountKey"]]
AccHier = AccHier.rename(columns={"AccountKey":"ParentAccountKey","ParentAccountKey":"ParentAccountKey2"})
AccHier.head()

,ParentAccountKey,ParentAccountKey2
0,1,NaN
1,2,1.0
2,3,2.0
3,4,3.0
4,5,3.0


In [29]:
#Acrescentando hierarquias em colunas
DimAccount_trat2 = DimAccount_trat.merge(AccHier, how='left', on='ParentAccountKey')
DimAccount_trat2.head()

,AccountKey,ParentAccountKey,AccountDescription,AccountType,Operator,ValueType,ParentAccountKey2
0,1,NaN,Balance Sheet,None,~,Currency,NaN
1,2,1.0,Assets,Assets,+,Currency,NaN
2,3,2.0,Current Assets,Assets,+,Currency,1.0
3,4,3.0,Cash,Assets,+,Currency,2.0
4,5,3.0,Receivables,Assets,+,Currency,2.0


In [30]:
#Acrescentando hierarquias em colunas
AccHier = AccHier.rename(columns={"ParentAccountKey":"ParentAccountKey2","ParentAccountKey2":"ParentAccountKey3"})
DimAccount_trat3 = DimAccount_trat2.merge(AccHier, how='left', on='ParentAccountKey2')
DimAccount_trat3.head()

,AccountKey,ParentAccountKey,AccountDescription,AccountType,Operator,ValueType,ParentAccountKey2,ParentAccountKey3
0,1,NaN,Balance Sheet,None,~,Currency,NaN,NaN
1,2,1.0,Assets,Assets,+,Currency,NaN,NaN
2,3,2.0,Current Assets,Assets,+,Currency,1.0,NaN
3,4,3.0,Cash,Assets,+,Currency,2.0,1.0
4,5,3.0,Receivables,Assets,+,Currency,2.0,1.0


In [31]:
#Acrescentando hierarquias em colunas
AccHier = AccHier.rename(columns={"ParentAccountKey2":"ParentAccountKey3","ParentAccountKey3":"ParentAccountKey4"})
DimAccount_trat4 = DimAccount_trat3.merge(AccHier, how='left', on='ParentAccountKey3')
DimAccount_trat4.head()

,AccountKey,ParentAccountKey,AccountDescription,AccountType,Operator,ValueType,ParentAccountKey2,ParentAccountKey3,ParentAccountKey4
0,1,NaN,Balance Sheet,None,~,Currency,NaN,NaN,NaN
1,2,1.0,Assets,Assets,+,Currency,NaN,NaN,NaN
2,3,2.0,Current Assets,Assets,+,Currency,1.0,NaN,NaN
3,4,3.0,Cash,Assets,+,Currency,2.0,1.0,NaN
4,5,3.0,Receivables,Assets,+,Currency,2.0,1.0,NaN


In [32]:
#Acrescentando hierarquias em colunas
AccHier = AccHier.rename(columns={"ParentAccountKey3":"ParentAccountKey4","ParentAccountKey4":"ParentAccountKey5"})
DimAccount_trat5 = DimAccount_trat4.merge(AccHier, how='left', on='ParentAccountKey4')
DimAccount_trat5.head()

,AccountKey,ParentAccountKey,AccountDescription,AccountType,Operator,ValueType,ParentAccountKey2,ParentAccountKey3,ParentAccountKey4,ParentAccountKey5
0,1,NaN,Balance Sheet,None,~,Currency,NaN,NaN,NaN,NaN
1,2,1.0,Assets,Assets,+,Currency,NaN,NaN,NaN,NaN
2,3,2.0,Current Assets,Assets,+,Currency,1.0,NaN,NaN,NaN
3,4,3.0,Cash,Assets,+,Currency,2.0,1.0,NaN,NaN
4,5,3.0,Receivables,Assets,+,Currency,2.0,1.0,NaN,NaN


In [33]:
#Identificar quais codigos de org tem na fato
regAcc = FactFinance_trat2.AccountKey.unique()
#Filtrando registros da Organization que tem na fato
DimAccount_trat6 = DimAccount_trat5[DimAccount_trat5['AccountKey'].isin(regAcc)]
DimAccount_trat7 = DimAccount_trat6[["AccountKey","AccountDescription","ParentAccountKey","ParentAccountKey2","ParentAccountKey3","ParentAccountKey4","ParentAccountKey5","AccountType"]]
DimAccount_trat7

,AccountKey,AccountDescription,ParentAccountKey,ParentAccountKey2,ParentAccountKey3,ParentAccountKey4,ParentAccountKey5,AccountType
3,4,Cash,3.0,2.0,1.0,NaN,NaN,Assets
5,6,Trade Receivables,5.0,3.0,2.0,1.0,NaN,Assets
6,7,Other Receivables,5.0,3.0,2.0,1.0,NaN,Assets
7,8,Allowance for Bad Debt,3.0,2.0,1.0,NaN,NaN,Assets
9,10,Raw Materials,9.0,3.0,2.0,1.0,NaN,Assets
...,...,...,...,...,...,...,...,...
89,92,Other Income,88.0,47.0,NaN,NaN,NaN,Revenue
90,93,Curr Xchg Gain/(Loss),88.0,47.0,NaN,NaN,NaN,Revenue
91,94,Taxes,47.0,NaN,NaN,NaN,NaN,Expenditures
97,100,Current Installments of Long-term Debt,27.0,26.0,25.0,1.0,NaN,Liabilities


In [34]:
#Separando os registros para ajuste de hierarquia
AccAux6 = DimAccount_trat7[DimAccount_trat7['ParentAccountKey5'].notna()]
AccAux6 = AccAux6.rename(columns={"AccountDescription":"AccountDescriptionNv6","ParentAccountKey":"AccountDescriptionNv5","ParentAccountKey2":"AccountDescriptionNv4","ParentAccountKey3":"AccountDescriptionNv3","ParentAccountKey4":"AccountDescriptionNv2","ParentAccountKey5":"AccountDescriptionNv1"})
AccAux6.head()

,AccountKey,AccountDescriptionNv6,AccountDescriptionNv5,AccountDescriptionNv4,AccountDescriptionNv3,AccountDescriptionNv2,AccountDescriptionNv1,AccountType
29,31,Salary & Other Comp,30.0,27.0,26.0,25.0,1.0,Liabilities
30,32,Insurance,30.0,27.0,26.0,25.0,1.0,Liabilities
31,33,Warranties,30.0,27.0,26.0,25.0,1.0,Liabilities
50,52,Intercompany Sales,51.0,50.0,49.0,48.0,47.0,Revenue
98,101,Trade Sales,51.0,50.0,49.0,48.0,47.0,Revenue


In [35]:
#Separando os registros para ajuste de hierarquia
AccAux5 = DimAccount_trat7[DimAccount_trat7['ParentAccountKey5'].isna() & DimAccount_trat7['ParentAccountKey4'].notna()]
AccAux5 = AccAux5.rename(columns={"AccountDescription":"AccountDescriptionNv5","ParentAccountKey":"AccountDescriptionNv4","ParentAccountKey2":"AccountDescriptionNv3","ParentAccountKey3":"AccountDescriptionNv2","ParentAccountKey4":"AccountDescriptionNv1","ParentAccountKey5":"AccountDescriptionNv6"})
AccAux5.head()

,AccountKey,AccountDescriptionNv5,AccountDescriptionNv4,AccountDescriptionNv3,AccountDescriptionNv2,AccountDescriptionNv1,AccountDescriptionNv6,AccountType
5,6,Trade Receivables,5.0,3.0,2.0,1.0,NaN,Assets
6,7,Other Receivables,5.0,3.0,2.0,1.0,NaN,Assets
9,10,Raw Materials,9.0,3.0,2.0,1.0,NaN,Assets
10,11,Work in Process,9.0,3.0,2.0,1.0,NaN,Assets
11,12,Finished Goods,9.0,3.0,2.0,1.0,NaN,Assets


In [36]:
#Separando os registros para ajuste de hierarquia
AccAux4 = DimAccount_trat7[DimAccount_trat7['ParentAccountKey4'].isna() & DimAccount_trat7['ParentAccountKey3'].notna()]
AccAux4 = AccAux4.rename(columns={"AccountDescription":"AccountDescriptionNv4","ParentAccountKey":"AccountDescriptionNv3","ParentAccountKey2":"AccountDescriptionNv2","ParentAccountKey3":"AccountDescriptionNv1","ParentAccountKey4":"AccountDescriptionNv6","ParentAccountKey5":"AccountDescriptionNv5"})
AccAux4.head()

,AccountKey,AccountDescriptionNv4,AccountDescriptionNv3,AccountDescriptionNv2,AccountDescriptionNv1,AccountDescriptionNv6,AccountDescriptionNv5,AccountType
3,4,Cash,3.0,2.0,1.0,NaN,NaN,Assets
7,8,Allowance for Bad Debt,3.0,2.0,1.0,NaN,NaN,Assets
12,13,Deferred Taxes,3.0,2.0,1.0,NaN,NaN,Assets
13,14,Prepaid Expenses,3.0,2.0,1.0,NaN,NaN,Assets
14,15,Intercompany Receivables,3.0,2.0,1.0,NaN,NaN,Assets


In [37]:
#Separando os registros para ajuste de hierarquia
AccAux3 = DimAccount_trat7[DimAccount_trat7['ParentAccountKey3'].isna() & DimAccount_trat7['ParentAccountKey2'].notna()]
AccAux3 = AccAux3.rename(columns={"AccountDescription":"AccountDescriptionNv3","ParentAccountKey":"AccountDescriptionNv2","ParentAccountKey2":"AccountDescriptionNv1","ParentAccountKey3":"AccountDescriptionNv6","ParentAccountKey4":"AccountDescriptionNv5","ParentAccountKey5":"AccountDescriptionNv4"})
AccAux3.head()

,AccountKey,AccountDescriptionNv3,AccountDescriptionNv2,AccountDescriptionNv1,AccountDescriptionNv6,AccountDescriptionNv5,AccountDescriptionNv4,AccountType
22,24,Other Assets,2.0,1.0,NaN,NaN,NaN,Assets
86,89,Interest Income,88.0,47.0,NaN,NaN,NaN,Revenue
87,90,Interest Expense,88.0,47.0,NaN,NaN,NaN,Expenditures
88,91,Gain/Loss on Sales of Asset,88.0,47.0,NaN,NaN,NaN,Revenue
89,92,Other Income,88.0,47.0,NaN,NaN,NaN,Revenue


In [38]:
#Separando os registros para ajuste de hierarquia
AccAux2 = DimAccount_trat7[DimAccount_trat7['ParentAccountKey2'].isna() & DimAccount_trat7['ParentAccountKey'].notna()]
AccAux2 = AccAux2.rename(columns={"AccountDescription":"AccountDescriptionNv2","ParentAccountKey":"AccountDescriptionNv1","ParentAccountKey2":"AccountDescriptionNv6","ParentAccountKey3":"AccountDescriptionNv5","ParentAccountKey4":"AccountDescriptionNv4","ParentAccountKey5":"AccountDescriptionNv3"})
AccAux2.head()

,AccountKey,AccountDescriptionNv2,AccountDescriptionNv1,AccountDescriptionNv6,AccountDescriptionNv5,AccountDescriptionNv4,AccountDescriptionNv3,AccountType
91,94,Taxes,47.0,NaN,NaN,NaN,NaN,Expenditures


In [39]:
#Junção dos registros
DimAccount_trat8 = pd.concat([AccAux6,AccAux5,AccAux4,AccAux3,AccAux2])
DimAccount_trat8

,AccountKey,AccountDescriptionNv6,AccountDescriptionNv5,AccountDescriptionNv4,AccountDescriptionNv3,AccountDescriptionNv2,AccountDescriptionNv1,AccountType
29,31,Salary & Other Comp,30.0,27.0,26.0,25.0,1.0,Liabilities
30,32,Insurance,30.0,27.0,26.0,25.0,1.0,Liabilities
31,33,Warranties,30.0,27.0,26.0,25.0,1.0,Liabilities
50,52,Intercompany Sales,51.0,50.0,49.0,48.0,47.0,Revenue
98,101,Trade Sales,51.0,50.0,49.0,48.0,47.0,Revenue
...,...,...,...,...,...,...,...,...
87,90,NaN,NaN,NaN,Interest Expense,88.0,47.0,Expenditures
88,91,NaN,NaN,NaN,Gain/Loss on Sales of Asset,88.0,47.0,Revenue
89,92,NaN,NaN,NaN,Other Income,88.0,47.0,Revenue
90,93,NaN,NaN,NaN,Curr Xchg Gain/(Loss),88.0,47.0,Revenue


In [40]:
#Substituindo os códigos pelos nomes
AccName = pd.Series(DimAccount.AccountDescription.values,index=DimAccount.AccountKey).to_dict()
DimAccount_trat9 = DimAccount_trat8.replace({"AccountDescriptionNv6": AccName,"AccountDescriptionNv5": AccName,"AccountDescriptionNv4": AccName,"AccountDescriptionNv3": AccName,"AccountDescriptionNv2": AccName,"AccountDescriptionNv1": AccName})
DimAccount_trat9

,AccountKey,AccountDescriptionNv6,AccountDescriptionNv5,AccountDescriptionNv4,AccountDescriptionNv3,AccountDescriptionNv2,AccountDescriptionNv1,AccountType
29,31,Salary & Other Comp,Accrued Expenses,Current Liabilities,Liabilities,Liabilities and Owners Equity,Balance Sheet,Liabilities
30,32,Insurance,Accrued Expenses,Current Liabilities,Liabilities,Liabilities and Owners Equity,Balance Sheet,Liabilities
31,33,Warranties,Accrued Expenses,Current Liabilities,Liabilities,Liabilities and Owners Equity,Balance Sheet,Liabilities
50,52,Intercompany Sales,Gross Sales,Net Sales,Gross Margin,Operating Profit,Net Income,Revenue
98,101,Trade Sales,Gross Sales,Net Sales,Gross Margin,Operating Profit,Net Income,Revenue
...,...,...,...,...,...,...,...,...
87,90,NaN,NaN,NaN,Interest Expense,Other Income and Expense,Net Income,Expenditures
88,91,NaN,NaN,NaN,Gain/Loss on Sales of Asset,Other Income and Expense,Net Income,Revenue
89,92,NaN,NaN,NaN,Other Income,Other Income and Expense,Net Income,Revenue
90,93,NaN,NaN,NaN,Curr Xchg Gain/(Loss),Other Income and Expense,Net Income,Revenue


## DimScenario

In [41]:
DimScenario.head()

,ScenarioKey,ScenarioName
0,1,Actual
1,2,Budget
2,3,Forecast


### Ter a informações contidas na tabela DimScenario como uma tabela separada é desnecessário e irei levar para a FactFinance

In [42]:
FactFinance_trat3 = pd.merge(FactFinance_trat2, DimScenario, how='left', on = 'ScenarioKey')

## FactFinance

In [43]:
FactFinance_trat3.head()

,FinanceKey,DateKey,OrganizationKey,DepartmentGroupKey,ScenarioKey,AccountKey,Amount,Date,ScenarioName
0,1,20101229,3,1,1,60,22080.0,2010-12-29,Actual
1,2,20101229,3,1,2,60,20200.0,2010-12-29,Budget
2,3,20101229,3,1,2,61,2000.0,2010-12-29,Budget
3,4,20101229,3,1,1,61,2208.0,2010-12-29,Actual
4,5,20101229,3,1,1,62,1546.0,2010-12-29,Actual


In [44]:
#Seleção das colunas
FactFinance_trat4 = FactFinance_trat3[["DateKey","OrganizationKey","DepartmentGroupKey","AccountKey","Amount","ScenarioName"]]
FactFinance_trat4.head()

,DateKey,OrganizationKey,DepartmentGroupKey,AccountKey,Amount,ScenarioName
0,20101229,3,1,60,22080.0,Actual
1,20101229,3,1,60,20200.0,Budget
2,20101229,3,1,61,2000.0,Budget
3,20101229,3,1,61,2208.0,Actual
4,20101229,3,1,62,1546.0,Actual


### DimDate

In [45]:
DimDate.head()

,DateKey,FullDateAlternateKey,DayNumberOfWeek,EnglishDayNameOfWeek,SpanishDayNameOfWeek,FrenchDayNameOfWeek,DayNumberOfMonth,DayNumberOfYear,WeekNumberOfYear,EnglishMonthName,SpanishMonthName,FrenchMonthName,MonthNumberOfYear,CalendarQuarter,CalendarYear,CalendarSemester,FiscalQuarter,FiscalYear,FiscalSemester
0,20050101,2005-01-01,7,Saturday,Sábado,Samedi,1,1,1,January,Enero,Janvier,1,1,2005,1,3,2005,2
1,20050102,2005-01-02,1,Sunday,Domingo,Dimanche,2,2,2,January,Enero,Janvier,1,1,2005,1,3,2005,2
2,20050103,2005-01-03,2,Monday,Lunes,Lundi,3,3,2,January,Enero,Janvier,1,1,2005,1,3,2005,2
3,20050104,2005-01-04,3,Tuesday,Martes,Mardi,4,4,2,January,Enero,Janvier,1,1,2005,1,3,2005,2
4,20050105,2005-01-05,4,Wednesday,Miércoles,Mercredi,5,5,2,January,Enero,Janvier,1,1,2005,1,3,2005,2


In [46]:
#Seleção das colunas, desconsiderando nomes em espanhol e frances
DimDate_trat = DimDate[["DateKey","FullDateAlternateKey","DayNumberOfWeek","EnglishDayNameOfWeek","DayNumberOfMonth","DayNumberOfYear","WeekNumberOfYear","EnglishMonthName","MonthNumberOfYear","CalendarQuarter","CalendarYear","CalendarSemester","FiscalQuarter","FiscalYear","FiscalSemester"]]
DimDate_trat.head()

,DateKey,FullDateAlternateKey,DayNumberOfWeek,EnglishDayNameOfWeek,DayNumberOfMonth,DayNumberOfYear,WeekNumberOfYear,EnglishMonthName,MonthNumberOfYear,CalendarQuarter,CalendarYear,CalendarSemester,FiscalQuarter,FiscalYear,FiscalSemester
0,20050101,2005-01-01,7,Saturday,1,1,1,January,1,1,2005,1,3,2005,2
1,20050102,2005-01-02,1,Sunday,2,2,2,January,1,1,2005,1,3,2005,2
2,20050103,2005-01-03,2,Monday,3,3,2,January,1,1,2005,1,3,2005,2
3,20050104,2005-01-04,3,Tuesday,4,4,2,January,1,1,2005,1,3,2005,2
4,20050105,2005-01-05,4,Wednesday,5,5,2,January,1,1,2005,1,3,2005,2


In [47]:
#Renomeação das colunas
DimDate_trat2 = DimDate_trat.rename(columns={"EnglishDayNameOfWeek":"DayNameOfWeek","EnglishMonthName":"MonthName"})
DimDate_trat2.head()

,DateKey,FullDateAlternateKey,DayNumberOfWeek,DayNameOfWeek,DayNumberOfMonth,DayNumberOfYear,WeekNumberOfYear,MonthName,MonthNumberOfYear,CalendarQuarter,CalendarYear,CalendarSemester,FiscalQuarter,FiscalYear,FiscalSemester
0,20050101,2005-01-01,7,Saturday,1,1,1,January,1,1,2005,1,3,2005,2
1,20050102,2005-01-02,1,Sunday,2,2,2,January,1,1,2005,1,3,2005,2
2,20050103,2005-01-03,2,Monday,3,3,2,January,1,1,2005,1,3,2005,2
3,20050104,2005-01-04,3,Tuesday,4,4,2,January,1,1,2005,1,3,2005,2
4,20050105,2005-01-05,4,Wednesday,5,5,2,January,1,1,2005,1,3,2005,2


## Escrita no banco de dados

In [48]:
#Criação de conexão ao banco de dados para leitura
server_name = "DESKTOP-9RK4J8O"
db_name = "AdventureWorksTrat"
connection_str =  'Driver={SQL Server};' + 'Server=' + server_name + ";Database=" + db_name + ";Trusted_Connection=yes;"

connection_url = URL.create("mssql+pyodbc", query={"odbc_connect": connection_str})
engine = create_engine(connection_url)
con = engine.connect()

In [49]:
#Escrever as tabelas
FactFinance_trat4.to_sql("FactFinance", con, index = False,if_exists='replace')
DimAccount_trat9.to_sql("DimAccount", con, index = False,if_exists='replace')
DimDate_trat2.to_sql("DimDate", con, index = False,if_exists='replace')
DimDepartmentGroup_trat3.to_sql("DimDepartmentGroup", con, index = False,if_exists='replace')
DimOrganization_trat7.to_sql("DimOrganization", con, index = False,if_exists='replace')

-1